<a href="https://colab.research.google.com/github/DJCordhose/llm-ops/blob/main/Present_Lllama_3.1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Lingual Medical Assessment Classification

## LLama 3.1 8B
https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct

### Measurements

#### 4 Bit T4
* Negatives: 35 s
* Positives: 38 s

#### 4 Bit T4 (optimized config)
* Negatives: 24 s
* Positives: 25 s

#### 8 Bit T4
* Negatives: 23 s
* Positives: 23 s

#### Full Res L4
* Negatives: 12.5 s
* Positives: 12.5 s

## Phi 3.5 MoE
* https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/discover-the-new-multi-lingual-high-quality-phi-3-5-slms/ba-p/4225280
* https://huggingface.co/microsoft/Phi-3.5-mini-instruct
* https://huggingface.co/microsoft/Phi-3.5-MoE-instruct

### Measurements

#### 4 Bit A100
* Negatives: 50 s
* Positives: 50 s

## Memory consumption and Quantization

### Quantization 4-Bit / 8-Bit
* 4-Bit (FP4): https://huggingface.co/blog/4bit-transformers-bitsandbytes
  * computation is not done in 4bit, the weights and activations are compressed to that format and the computation is still kept in native dtype
* 8-Bit (LLM.int8()): https://huggingface.co/blog/hf-bitsandbytes-integration#a-gentle-summary-of-llmint8-zero-degradation-matrix-multiplication-for-large-language-models
  * performs the matrix multiplication of the outliers in FP16 and the non-outliers in int8

### Memory consumption for Lllama 3.1 models
- How much memory does Llama 3.1 need for weights and cache (depending on the actually used context length) https://huggingface.co/blog/llama31#inference-memory-requirements
- Detailed, but still comprehensive explanation of how inference in LLMs works: https://developer.nvidia.com/blog/mastering-llm-techniques-inference-optimization/


# Hands-on: Inference on commodity hardware

**Run the assessment classification on the best Llama 3.x model possible**

1. Get a very subjective impression: Do you like the results in terms of e.g.
   1. Quality of language
   1. Halluzination
   1. Accuracy
   1. Speed
   1. Memory consumption
1. If more than one Llama 3.x model actually runs, subjectively compare the results

## Optional
1. Compare to Phi Mini
1. Try a different langauge (German is prepared)


In [1]:
%%time

!pip install --upgrade -q transformers accelerate torch bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.7 MB/s eta 0:00:00
CPU times: user 110 ms, sys: 17 ms, total: 127 ms
Wall time: 17.6 s


In [2]:
# Can take forever, and we can do without it

# !pip install --upgrade flash_attn

In [3]:
import transformers

transformers.__version__

'4.46.0'

In [12]:
from google.colab import userdata

In [13]:
# Configure HuggingFace token as a Colab Secret, use key symbol on the left panel
!huggingface-cli login --token {userdata.get('HF_TOKEN')}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
from IPython.display import Markdown

In [16]:
!nvidia-smi

Sun Oct 27 21:40:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   52C    P8              13W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
# kind = 'Lllama_3.1_8B_4bit'
# kind = 'Lllama_3.1_8B_8bit'
kind = 'Lllama_3.1_8B_16bit'
# kind = 'Lllama_3.2_3B_16bit'
# kind = 'Lllama_3.2_1B_16bit'
# kind = 'Phi-3.5-MoE_4bit'
# kind = "Phi-3.5-mini_16bit"

# lang = "de"
lang = "en"

In [18]:
if "Lllama_3.1_8B" in kind:
  model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
elif "Lllama_3.2_3B" in kind:
  model_id = "meta-llama/Llama-3.2-3B-Instruct"
elif "Lllama_3.2_1B" in kind:
  model_id = "meta-llama/Llama-3.2-1B-Instruct"
elif "Phi-3.5-MoE" in kind:
  model_id = "microsoft/Phi-3.5-MoE-instruct"
else:
  model_id = "microsoft/Phi-3.5-mini-instruct"

print(model_id)

meta-llama/Meta-Llama-3.1-8B-Instruct


In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

***note:*** execute in a termial 'watch -n 0.5 nvidia-smi' to see the GPU usage and when the model is loaded onto it

In [20]:
%%time

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch_dtype = None
quantization_config = None


if "8bit" in kind:
  print("Using 8Bit quantization")
  quantization_config = BitsAndBytesConfig(load_in_8bit=True)
elif "4bit" in kind:
  print("Using 4Bit quantization")
  # quantization_config = BitsAndBytesConfig(load_in_4bit=True)
  quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
  )
else:
  print("Using Full Resolution")
  torch_dtype = torch.bfloat16

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    torch_dtype=torch_dtype,
    device_map="cuda",
    trust_remote_code=True
)

Using Full Resolution


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

CPU times: user 33.2 s, sys: 26.4 s, total: 59.6 s
Wall time: 6min 39s


In [21]:
!nvidia-smi

Sun Oct 27 21:47:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   47C    P0              29W /  72W |  15637MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [22]:
positive_en = [
  "With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.",
  "The socio-medical prerequisites for the prescribed aid supply have been met.",
  "Everyday relevant usage benefits have been determined.",
  "Socio-medical indication for the aid is confirmed.",
  "Contraindications have been excluded; there are no contraindications for the use of the requested aid."
]

In [23]:
negative_en = [
  "No specific findings can be derived from the diagnosis currently named as the basis for the regulation.",
  "According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.",
  "A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appropriate and therefore necessary has not been transmitted.",
  "From an overall view of the information available here, it cannot be seen how the supply of the insured with the product could be justified, nor can the safety of such a supply be confirmed.",
  "A medical justification for why a product not listed in the directory of aids should be used in the present case has not been transmitted."
]

In [24]:
positive_de = [
  "Bei der hier benannten Diagnose ist das Erfordernis eines Ausgleichs zur Sicherstellung des Grundbedürfnisses denkbar.",
  "Die sozialmedizinischen Voraussetzungen für die verordnete Hilfsmittelversorgung sind erfüllt.",
  "Alltagsrelevante Gebrauchsvorteile werden festgestellt.",
  "Sozialmedizinische Indikation für das Hilfsmittel wird bestätigt.",
  "Kontraindikationen wurden ausgeschlossen, es liegen keine Gegenanzeigen für die Verwendung des beantragten Hilfsmittels vor."
]

In [25]:
negative_de = [
  "Aus der aktuell als verordnungsbegründend benannten Diagnose lässt sich kein konkreter Befund ableiten.",
  "Gemäß den Leistungsauszügen der Krankenkasse ist der Versicherte bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkt versorgt.",
  "Eine medizinisch nachvollziehbare Begründung, weshalb der Einsatz einer befundadäquaten orthopädietechnischen Hilfsmittelversorgung nicht ausreichend und stattdessen eine elektrische Fußheberstimulation zum Gehen zweckmäßiger und deshalb notwendig wäre, wurde nicht übermittelt.",
  "In der Gesamtschau der hier vorliegenden Informationen kann nicht erkannt werden, wie die Versorgung des Versicherten mit dem Produkt begründet werden könnte, noch kann die Unbedenklichkeit einer solchen Versorgung bestätigt werden.",
  "Eine ärztliche Begründung, warum im vorliegenden Fall ein nicht im Hilfsmittelverzeichnis gelistetes Produkt zum Einsatz kommen soll, wird nicht übermittelt."
]

In [26]:

if lang == "de":
  negative = negative_de
  positive = positive_de
else:
  negative = negative_en
  positive = positive_en

In [27]:
idx = 0
# assessment = negative[idx]
assessment = positive[idx]

assessment

'With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.'

In [28]:
%%time

if lang == "de":
  messages = [
    {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
    {"role": "user", "content": f'''
    Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

    # Assessment
    {assessment}

    '''}
  ]
else:
  messages = [
    {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
    {"role": "user", "content": f'''
What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

# Assessment
{assessment}

'''}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False
)
response = outputs[0][input_ids.shape[-1]:]
Markdown(tokenizer.decode(response, skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


CPU times: user 3.51 s, sys: 262 ms, total: 3.78 s
Wall time: 4.42 s


No

The assessment suggests that the need for compensation to ensure the basic need is conceivable, implying that the situation is uncertain or open to interpretation. A positive or negative recommendation cannot be conclusively given based on this information.

In [29]:
def eval_assessment(assessment):
  if lang == "de":
    yes = "Ja"
    no = "Nein"
    messages = [
  {"role": "system", "content": "Du bist ein kompetenter Experte auf dem Gebiet der gesetzlichen Krankenversicherung und sprichst Deutsch. Antworte präzise, ernst und formell."},
  {"role": "user", "content": f'''
  Was ist das Ergebnis der Bewertung? Wird eine positive oder negative Empfehlung gegeben? Antworte mit 'Ja' oder 'Nein' und gib anschließend eine sehr kurze Begründung für die Einschätzung."

  # Assessment
  {assessment}

  '''}
  ]
  else:
    yes = "Yes"
    no = "No"
    messages = [
      {"role": "system", "content": "You are an English-speaking, competent expert in the field of statutory health insurance. Answer consice, serious and formal."},
      {"role": "user", "content": f'''
  What is the result of the assessment? Is a positive or negative recommendation given? Answer with "Yes" or "No" and then provide a brief justification for your assessment.

  # Assessment
  {assessment}

  '''}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=512,
      eos_token_id=terminators,
      pad_token_id=tokenizer.eos_token_id,
      do_sample=False
  )
  response = outputs[0][input_ids.shape[-1]:]
  result = tokenizer.decode(response, skip_special_tokens=True)
  if result.startswith(yes):
    return "Positive", result
  elif result.startswith(no):
    return "Negative", result
  else:
    return "Neutral", result

## Negative

In [30]:
%%time

negative_results = []
negative_explanations = []

for assessment in negative:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  negative_results.append(result)
  negative_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: No specific findings can be derived from the diagnosis currently named as the basis for the regulation.
Negative: No

The assessment indicates that there are no specific findings that can be derived from the diagnosis, implying that the current diagnosis does not provide sufficient information to make a recommendation.
-----
Assessment: According to the service extracts from the health insurance, the insured has already been provided with the functional product requested according to its area of application.
Negative: No

The assessment indicates that the insured has already received the requested functional product, implying that the need for it has been met. Therefore, a negative recommendation is given, as there is no further requirement for the product.
-----
Assessment: A medically comprehensible explanation as to why the use of an orthopedic aid corresponding to the findings is not sufficient and instead electric foot lifter stimulation for walking would be more appro

## Positive

In [31]:
%%time

positive_results = []
positive_explanations = []

for assessment in positive:
  print(f"Assessment: {assessment}")
  result, explanation = eval_assessment(assessment)
  positive_results.append(result)
  positive_explanations.append(explanation)
  print(f"{result}: {explanation}")
  print("-----")

Assessment: With the diagnosis named here, the need for compensation to ensure the basic need is conceivable.
Negative: No

The assessment suggests that the need for compensation to ensure the basic need is conceivable, implying that the situation may warrant some form of financial support. However, it does not explicitly state that compensation is necessary or justified, which would typically be the case for a positive recommendation.
-----
Assessment: The socio-medical prerequisites for the prescribed aid supply have been met.
Positive: Yes

The socio-medical prerequisites for the prescribed aid supply have been met, indicating that the individual's medical and social situation justify the provision of the aid. This suggests a positive assessment and recommendation for the aid supply.
-----
Assessment: Everyday relevant usage benefits have been determined.
Negative: No

The assessment indicates that the everyday relevant usage benefits have been determined, but it does not provide in

In [32]:
!nvidia-smi

Sun Oct 27 21:48:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   62C    P0              72W /  72W |  15733MiB / 23034MiB |     87%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [33]:
import pandas as pd

df = pd.DataFrame({
    'assesment': negative + positive,
    'y_true': ['Negative'] * len(negative) + ['Positive'] * len(positive),
    'y_hat': negative_results + positive_results,
    'explanation': negative_explanations + positive_explanations
})
df

,assesment,y_true,y_hat,explanation
0,No specific findings can be derived from the d...,Negative,Negative,No\n\nThe assessment indicates that there are ...
1,According to the service extracts from the hea...,Negative,Negative,No\n\nThe assessment indicates that the insure...
2,A medically comprehensible explanation as to w...,Negative,Negative,No\n\nThe assessment indicates that a medicall...
3,From an overall view of the information availa...,Negative,Negative,No\n\nThe assessment indicates that the supply...
4,A medical justification for why a product not ...,Negative,Negative,No\n\nA medical justification is required to j...
5,"With the diagnosis named here, the need for co...",Positive,Negative,No\n\nThe assessment suggests that the need fo...
6,The socio-medical prerequisites for the prescr...,Positive,Positive,Yes\n\nThe socio-medical prerequisites for the...
7,Everyday relevant usage benefits have been det...,Positive,Negative,No\n\nThe assessment indicates that the everyd...
8,Socio-medical indication for the aid is confir...,Positive,Positive,Yes\n\nThe assessment confirms a socio-medical...
9,Contraindications have been excluded; there ar...,Positive,Positive,Yes\n\nThe assessment indicates that the reque...


In [34]:
df.to_excel(f'results_{kind}_{lang}.xlsx', index=False)

In [35]:
!ls -l

total 12
-rw-r--r-- 1 root root 6359 Oct 27 21:48 results_Lllama_3.1_8B_16bit_en.xlsx
drwxr-xr-x 1 root root 4096 Oct 24 13:20 sample_data
